In [1]:
import numpy as np
import pandas as pd
import csv
import cudf
import cuml
import cupy
from cuml.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder

# KNN Predictions

In [2]:
data = pd.read_csv('../input/amazon-clean-dataset2/data.csv')
y = np.array(data['BROWSE_NODE_ID']).astype(np.int)
del data

le = LabelEncoder()
y = le.fit_transform(y)

d1 = np.load('../input/pretrained-bert-embedding-1th-part-new-model/data_embeddings1.npy').astype(np.float16)
d2 = np.load('../input/pretrained-bert-embedding-2th-part-new-model/data_embeddings2.npy').astype(np.float16)
a = np.append(d1, d2, axis = 0)
del d1, d2

In [3]:
model = KNeighborsClassifier(n_neighbors = 15)
model.fit(a, y[ : a.shape[0]])
del a, y
t0 = np.load('../input/pretrained-bert-embedding-4th-part-new-model/test_embeddings.npy').astype(np.float16)
proba_knn = model.predict_proba(t0)
pred_knn = le.inverse_transform(model.predict(t0))
np.save('proba_knn.npy', proba_knn)

In [4]:
test = pd.read_csv('../input/amazon-ml-challenge-dataset/dataset/test.csv', escapechar = "\\"  , quoting = csv.QUOTE_NONE, usecols = ['PRODUCT_ID'])
test['BROWSE_NODE_ID'] = pred_knn
test.to_csv("submission_knn.csv", index = False)
test.head()

,PRODUCT_ID,BROWSE_NODE_ID
0,1,1140
1,2,15772
2,3,199530
3,4,125
4,5,7644
